### Questions

- $\gamma$ 確認(v)
- 資料集跟 paper 少部份不一樣(v)
- P1 解不出來...

### Solutions

- Check outlier(min, max)
- objective + soft margin(slack)
- non-linear kernel -> Gaussian kernel(range of $\gamma$)

In [76]:
from gurobipy import *
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, RobustScaler

### Data helper

In [77]:
import os
import pandas as pd
from ucimlrepo import fetch_ucirepo 

class DataHolder():
    def __init__(self) -> None:

        self.files_id = {
            "careval": 19,
            "wisconsin": 17,
            "votes": 105,
        }

    def get(self, name):

        if name == "nursery":
            df = self.fetch_dataset(name)
#             df = df.sample(frac=1)
            X = df.loc[:, df.columns[:-1]]
            X = self.encode(X)
            y = df[df.columns[-1]].to_frame('class')
            y = self.multiclass_preprocessing(y)

        elif name == "australian":
            df = self.fetch_dataset(name)
            X = df.loc[:, df.columns[:-1]]
            X['A1'] = X['A1'].replace({0: 'a', 1: 'b'})
            X['A4'] = X['A4'].replace({1: 'p', 2: 'g', 3: 'gg'})
            X['A5'] = X['A5'].replace({1: 'ff', 2: 'd', 3: 'i', 4: 'k', 5: 'j', 6: 'aa', 7: 'm', 8: 'c', 
                                        9: 'w', 10: 'e', 11: 'q', 12: 'r', 13: 'cc', 14: 'x'})
            X['A6'] = X['A6'].replace({1: 'ff', 2: 'dd', 3: 'j', 4: 'bb', 5: 'v', 6: 'n', 7: 'o', 8: 'h',
                                        9: 'z'})
            X['A8'] = X['A8'].replace({0: 'f', 1: 't'})
            X['A9'] = X['A9'].replace({0: 'f', 1: 't'})
            X['A11'] = X['A11'].replace({0: '0', 1: 't'})
            X['A12'] = X['A12'].replace({1: 's', 2: 'g', 3: 'p'})
            X = self.encode(X)
            y = df[df.columns[-1]].to_frame('class')
            
        elif name == "gastrointestinal":
            df = self.fetch_dataset(name)
            X = df.iloc[:, 3:]
            t1 = df.iloc[:, 0].to_frame('class')
            t2 = df.iloc[:, 1].to_frame('class')
            t3 = df.iloc[:, 2].to_frame('class')
            indices_with_WL = t3[t3['class'] == '1'].index
            indices_with_NBI = t3[t3['class'] == '2'].index
            result_t2 = t2.loc[indices_with_WL]
            X = X.loc[indices_with_WL]

            y = self.multiclass_preprocessing(result_t2)
            y = self.encode(y)
            
        elif name == "votes":

            df = self.fetch_dataset(name)
            X = df.iloc[:, 1:]
            y = df.iloc[:, 0]
            X = self.encode(X)
            y = self.encode(y)

        else:
                    
            df = self.fetch_dataset(name)
            
            X = df.data.features 
            y = df.data.targets 
            
            df = pd.concat([X,y],axis=1)
            df = df.sample(frac=1) # shuffle
            
            if name == 'careval':
                X = self.encode(X)
                y = self.multiclass_preprocessing(y)

            y = self.encode(y)

        return X, y
        
    def fetch_dataset(self, name):

        prefix = 'data'+os.sep

        if name == 'nursery':

            file_path = prefix+name+os.sep+name+'.data'
            df = pd.read_csv(file_path, header=None) 
            attr = ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health', 'class']
            df.columns = attr

        elif name == 'votes':

            file_path = prefix+name+os.sep+'house-votes-84.data'
            df = pd.read_csv(file_path, delimiter=',', header=None) 
            class_name = ['class']
            features_name = [f'A{i+1}' for i in range(len(df.columns)-1)]
            column_names = class_name+features_name
            df.columns = column_names

        elif name == 'australian':

            file_path = prefix+name+os.sep+'australian.dat'
            df = pd.read_csv(file_path, delimiter=' ', header=None) 
            attr = [f'A{i+1}' for i in range(15)]
            df.columns = attr
            
        elif name == 'gastrointestinal':

            file_path = prefix+name+os.sep+'data.txt'
            df = pd.read_csv(file_path, header=None)
            list_of_rows = df.values.tolist()
            raw_features = list_of_rows[3:]
            labels_name = [f'label{i+1}' for i in range(3)]
            features_name = [f'A{i+1}' for i in range(698)]
            attr = labels_name + features_name
            df = pd.DataFrame(list_of_rows).transpose()
            df.columns = attr
            
        else:

            df = fetch_ucirepo(id=self.files_id[name]) 

        return df
    
    def impute(self, df):
        columns_with_missing_values = df.data.features.columns[df.data.features.isna().any()].tolist()

        # imputation with mode
        df.data.features.loc[:, columns_with_missing_values] = \
            df.data.features[columns_with_missing_values].fillna(df.data.features[columns_with_missing_values].mode().iloc[0])
        return df
    
    def encode(self, df):
        df = pd.get_dummies(df, drop_first=True)
        df = df.astype(int)
        return df
    
    def multiclass_preprocessing(self, y):
        
        majority_class = y['class'].mode().iloc[0]
        y_copy = y.copy()
        y_copy['binary_label'] = y_copy['class'].apply(lambda x: 1 if x == majority_class else 0)
        y_copy = y_copy.drop('class', axis=1)
        return y_copy
    
    def show_details(self, X, y):
        nums_of_row = X.shape[0]
        nums_of_feature = X.shape[1]
        positive_count = y.sum()
        print(f'nums_of_row:{nums_of_row}')
        print(f'nums_of_feature:{nums_of_feature}')
        if (positive_count.values < (nums_of_row - positive_count.values)):
            positive_count == nums_of_row - positive_count.values
        print(f'positive_count:{positive_count.values}')

if __name__ == '__main__':
    
    dh = DataHolder()
    # X, y = dh.get('gastrointestinal') # not yet 我通靈不到他怎麼算的
    # X, y = dh.get('nursery') # done
    X, y = dh.get('australian') # done -> positive count 307 vs. 383
    # X, y = dh.get('careval') # done 
    # X, y = dh.get('wisconsin') # done -> positive count 212 vs. 357
    # X, y = dh.get('votes') # done -> positive count 168 vs. 267 
    
    # min max
#     scaler = MinMaxScaler()
    scaler = RobustScaler()
    X[X.columns] = scaler.fit_transform(X[X.columns])
    
    dh.show_details(X,y)
#     X = X.replace(0, -1)
    y = y.replace(0, -1)
    
    
   
    
    
    

nums_of_row:690
nums_of_feature:34
positive_count:[307]


In [116]:
X = X[:500]
y = y[:500]

,class
2,-1
3,1


## P1

In [79]:
def P1(X:pd.DataFrame, y:np.ndarray, c:list = None, lambda_:list = [0.8, 0.8], M2:int = 100, M3:int = 100):
    """ The cost-sensitive FS procedure """
    """ The first constraint causes infeasible """
    """ Model is feasible if zeta is continuous """
    
    # set up
    N = X.shape[1]
    size = X.shape[0]   
    x = np.array([X.iloc[i, :] for i in range(size)])
    y = np.array(y).flatten()
    print(x.shape)
    if c == None:
        c = [1 for _ in range(N)]

    # create model
    model = Model("P1")
    
    # # decision variables
    # w = [model.addVar(vtype=GRB.CONTINUOUS) for _ in range(N)]
    # beta = model.addVar(vtype=GRB.CONTINUOUS)
    # zeta = [model.addVar(lb=0, ub=1, vtype=GRB.BINARY) for _ in range(size)]
    # z = [model.addVar(vtype=GRB.BINARY) for _ in range(N)]
    w = model.addVars(N, lb=float('-inf'), vtype=GRB.CONTINUOUS)
    beta = model.addVar(lb=float('-inf'), vtype=GRB.CONTINUOUS)
    zeta = model.addVars(size, vtype=GRB.BINARY)
    z = model.addVars(N, vtype=GRB.BINARY)
    
    
    # # constraints 
    # model.addConstrs(
    #     y[i] * (np.dot(np.array(w), x[i]) + beta) >= 1 - M2 * (1 - zeta[i]) for i in range(size)
    # )
    # model.addConstr(
    #     quicksum(zeta[i] * (1 - y[i]) for i in range(size)) >= lambda_[0] * quicksum(1 - y[i] for i in range(size))
    # )
    # model.addConstr(
    #     quicksum(zeta[i] * (1 + y[i]) for i in range(size)) >= lambda_[1] * quicksum(1 + y[i] for i in range(size))
    # )
    # model.addConstrs(
    #     w[k] <= M3 * z[k] for k in range(N)
    # )
    # model.addConstrs(
    #     w[k] >= -M3 * z[k] for k in range(N)
    # )
    model.addConstrs(y[i] * (quicksum(w[j]*x[i, j] for j in range(N)) + beta) >= 1 - M2 * (1 - zeta[i]) for i in range(size))
    model.addConstr(quicksum(zeta[i] * (1 - y[i]) for i in range(size)) >= lambda_[0] * quicksum(1 - y[i] for i in range(size)))
    model.addConstr(quicksum(zeta[i] * (1 + y[i]) for i in range(size)) >= lambda_[1] * quicksum(1 + y[i] for i in range(size)))
    model.addConstrs(w[k] <= M3 * z[k] for k in range(N))
    model.addConstrs(w[k] >= -M3 * z[k] for k in range(N))
    
    
    # objective function
    model.setObjective(quicksum(c[k] * z[k] for k in range(N)), GRB.MINIMIZE)
    
    # optimization
    model.optimize()
    
    # # result
    # result = {
    #     "w": [i.x for i in w],
    #     "beta": beta.x,
    #     "z": [i.x for i in z],
    #     "zeta": [i.x for i in zeta]
    # }
    result = {
        "w": [w[i].x for i in range(N)],
        "beta": beta.x,
        "z": [z[i].x for i in range(N)],
        "zeta": [zeta[i].x for i in range(size)]
    }
    
    # print(f'obj - {model.ObjVal}')
    print(f'obj: {model.ObjVal}')
    
    return result

In [80]:
result = P1(X,y,lambda_=[0.5,0.5])
res = P1(X,y)["z"]
print(result["beta"])
print(res.count(0),X.shape[1])
choose = [i for i in range(len(res)) if res[i] == 1]
choose

(500, 34)
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2413567 - for non-commercial use only - registered to b0___@ntu.edu.tw
Optimize a model with 570 rows, 569 columns and 5865 nonzeros
Model fingerprint: 0x6ac6a999
Variable types: 35 continuous, 534 integer (534 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 3e+02]
Found heuristic solution: objective 23.0000000
Presolve removed 4 rows and 2 columns
Presolve time: 0.01s
Presolved: 566 rows, 567 columns, 5857 nonzeros
Variable types: 33 continuous, 534 integer (534 binary)
Found heuristic solution: objective 16.0000000



Root relaxation: objective 0.000000e+00, 639 iterations, 0.01 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0  117   16.00000    0.00000   100%     -    0s
H    0     0                       5.0000000    0.00000   100%     -    0s
H    0     0                       3.0000000    0.00000   100%     -    0s
     0     0    0.00000    0  134    3.00000    0.00000   100%     -    0s
     0     0    0.00000    0  120    3.00000    0.00000   100%     -    0s
     0     0    0.00000    0  117    3.00000    0.00000   100%     -    0s
     0     0    0.00000    0  120    3.00000    0.00000   100%     -    0s
     0     0    0.00000    0  117    3.00000    0.00000   100%     -    0s
H    0     0                       2.0000000    0.00000   100%     -    0s
     0     0    0.00000    0  121    2.00000    0.00000   100%     -    0s
     0   

[29]

In [81]:
result = P1(X,y)

(500, 34)
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2413567 - for non-commercial use only - registered to b0___@ntu.edu.tw
Optimize a model with 570 rows, 569 columns and 5865 nonzeros
Model fingerprint: 0x1f95751f
Variable types: 35 continuous, 534 integer (534 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 4e+02]
Found heuristic solution: objective 14.0000000
Presolve removed 4 rows and 2 columns
Presolve time: 0.01s
Presolved: 566 rows, 567 columns, 5857 nonzeros
Variable types: 33 continuous, 534 integer (534 binary)
Found heuristic solution: objective 11.0000000

Root relaxation: objective 0.000000e+00, 848 iterations, 0.02 seconds (0.04 work units)

    Nodes 

In [82]:
result["zeta"].count(1), len(result["zeta"])

(421, 500)

In [83]:
result["beta"]

99.00000000000594

### 其實 P2, P3 可以只寫成一個，但我不確定會不會比較好

## P2

In [84]:
def P2(X:pd.DataFrame, y, zk:list=None, C:int=100, lambda_:list=[0.5,0.5], M1:int=100):
    """ Cost-sensitive sparse SVMs - linear """
    # set up
    N = X.shape[1]
    size = X.shape[0]   
    x = np.array([X.iloc[i, :] for i in range(size)])
    y = np.array(y).flatten()
    
    
    
    # z = [0 for _ in range(N)]
    # for i in zk:
    #     z[i] = 1
    z = [int(zi) for zi in zk]
    assert(len(zk) == N)
    for zi in z:
        assert(zi == 0 or zi == 1)
        
    # create model 
    model = Model("P2")
    
    # decision variables
    # w = [model.addVar(vtype=GRB.CONTINUOUS) for _ in range(N)]
    # beta = model.addVar(vtype=GRB.CONTINUOUS)
    # xi = [model.addVar(lb=0,vtype=GRB.CONTINUOUS) for _ in range(size)]
    # zeta = [model.addVar(vtype=GRB.BINARY) for _ in range(size)]
    w = model.addVars(N, lb=float('-inf'), vtype=GRB.CONTINUOUS)
    beta = model.addVar(lb=float('-inf'), vtype=GRB.CONTINUOUS)
    xi = model.addVars(size, vtype=GRB.CONTINUOUS)
    zeta = model.addVars(size, vtype=GRB.BINARY)
    
    # constraints
    # model.addConstrs(
    #     y[i] * (quicksum(w[j] * z[j] * x[i][j] for j in range(N)) + beta) >= 1 - xi[i] for i in range(size)
    # )
    # model.addConstrs(
    #     xi[i] <= M1 * (1 - zeta[i]) for i in range(size)
    # )
    # model.addConstr(
    #     quicksum(zeta[i] * (1 - y[i]) for i in range(size)) >= lambda_[0] * quicksum(1 - y[i] for i in range(size))
    # )
    # model.addConstr(
    #     quicksum(zeta[i] * (1 + y[i]) for i in range(size)) >= lambda_[1] * quicksum(1 + y[i] for i in range(size))
    # )
    model.addConstrs(y[i] * (quicksum(w[j] * z[j] * x[i,j] for j in range(N)) + beta) >= 1 - xi[i] for i in range(size))
    model.addConstrs(xi[i] <= M1 * (1 - zeta[i]) for i in range(size))
    model.addConstr(quicksum(zeta[i] * (1 - y[i]) for i in range(size)) >= lambda_[0] * quicksum(1 - y[i] for i in range(size)))
    model.addConstr(quicksum(zeta[i] * (1 + y[i]) for i in range(size)) >= lambda_[1] * quicksum(1 + y[i] for i in range(size)))
    
    # objective function
    
    model.setObjective(quicksum(w[j] ** 2 * z[j] for j in range(N)) + C * quicksum(xi[i] for i in range(size)), GRB.MINIMIZE)
    
    # optimization
    model.optimize()
    
    
    # result
    # result = {
    #     "w": [i.x for i in w],
    #     "beta": beta.x,
    #     "xi": [i.x for i in xi]
    # }
    result = {
        "w": [w[i].x for i in range(N)],
        "beta": beta.x,
        "xi": [xi[i].x for i in range(size)]
    }
    
    return result

In [85]:
# res = P2(X,y, choose)
res = P2(X,y, result["z"])

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2413567 - for non-commercial use only - registered to b0___@ntu.edu.tw
Optimize a model with 1002 rows, 1035 columns and 2737 nonzeros
Model fingerprint: 0xe8551ad9
Model has 1 quadratic objective term
Variable types: 535 continuous, 500 integer (500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+02, 1e+02]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective 40504.000000
Presolve removed 107 rows and 140 columns
Presolve time: 0.02s
Presolved: 895 rows, 895 columns, 2469 nonzeros
Presolved model has 1 quadratic objective terms
Variable types: 449 continuous, 446 integer (446 binary)

Root relaxation: objective 

In [86]:
res["w"]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.0,
 0.0,
 0.0,
 0.0,
 0.0]

## P3

In [87]:
def K_(z:list, gamma:float=0.5):
    N = len(z)
    def func(x,x_prime):
        """ gamma need to be tuned """
#         global gamma
        return np.exp(-gamma * sum(z[k] * (x[k] - x_prime[k]) ** 2 for k in range(N)))
    return func

In [88]:
def P3(X:pd.DataFrame, y, zk:list=None, C:int=5, gamma:float=0.5, lambda_:list=[0.1, 0.1], M1:int=100):
    # set up
    N = X.shape[1]
    size = X.shape[0]   
    x = np.array([X.iloc[i, :] for i in range(size)])
    y = np.array(y).flatten()

    # z = [0 for _ in range(N)]
    # for i in zk:
    #     z[i] = 1
    z = [int(zi) for zi in zk]
    assert(len(zk) == N)
    for zi in z:
        assert(zi == 0 or zi == 1)
    
    global K_
    K = K_(z, gamma)
    
    # create model
    model = Model("P3")
    
    # decision variables
    # alpha = [model.addVar(lb=0, ub=C/2, vtype=GRB.CONTINUOUS) for _ in range(size)]
    # xi = [model.addVar(lb=0, vtype=GRB.CONTINUOUS) for _ in range(size)]
    # zeta = [model.addVar(vtype=GRB.BINARY) for _ in range(size)]
    # beta = model.addVar(vtype=GRB.CONTINUOUS)
    alpha = model.addVars(size, ub=C/2, vtype=GRB.CONTINUOUS)
    xi = model.addVars(size, vtype=GRB.CONTINUOUS)
    zeta = model.addVars(size, vtype=GRB.BINARY)
    beta = model.addVar(lb=float('-inf'), vtype=GRB.CONTINUOUS)
    
    # constraints
    # model.addConstrs(
    #     (y[i] * (quicksum(alpha[j] * y[j] * K(x[i], x[j]) for j in range(size)) + beta) >= 1 - xi[i]) for i in range(size)
    # )
    # model.addConstrs(
    #     (xi[i] <= M1 * (1 - zeta[i])) for i in range(size)
    # )
    # model.addConstr(
    #     quicksum(alpha[i] * y[i] for i in range(size)) == 0
    # )
    # model.addConstr(
    #     quicksum(zeta[i] * (1 - y[i]) for i in range(size)) >= lambda_[0] * (quicksum(1 - y[i] for i in range(size)))
    # )
    # model.addConstr(
    #     quicksum(zeta[i] * (1 + y[i]) for i in range(size)) >= lambda_[1] * (quicksum(1 + y[i] for i in range(size)))
    # )
    model.addConstrs((y[i] * (quicksum(alpha[j] * y[j] * K(x[i], x[j]) for j in range(size)) + beta) >= 1 - xi[i]) for i in range(size))
    model.addConstrs((xi[i] <= M1 * (1 - zeta[i])) for i in range(size))
    model.addConstr(quicksum(alpha[i] * y[i] for i in range(size)) == 0)
    model.addConstr(quicksum(zeta[i] * (1 - y[i]) for i in range(size)) >= lambda_[0] * (quicksum(1 - y[i] for i in range(size))))
    model.addConstr(quicksum(zeta[i] * (1 + y[i]) for i in range(size)) >= lambda_[1] * (quicksum(1 + y[i] for i in range(size))))
    
    # optimization
    model.optimize()
    
    # result
    # result = {
    #     "alpha": [i.x for i in alpha],
    #     "xi": [i.x for i in xi],
    #     "zeta": [i.x for i in zeta],
    #     "beta": beta.x
    # }
    result = {
        "alpha": [alpha[i].x for i in range(size)],
        "xi": [xi[i].x for i in range(size)],
        "zeta": [zeta[i].x for i in range(size)],
        "beta": beta.x
    }
    
    return result

In [89]:
# P3(X, y, choose)
P3(X, y, result["z"])

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2413567 - for non-commercial use only - registered to b0___@ntu.edu.tw
Optimize a model with 1003 rows, 1501 columns and 253000 nonzeros
Model fingerprint: 0x1161cc7a
Variable types: 1001 continuous, 500 integer (500 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+02]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 501 rows and 999 columns
Presolve time: 0.10s
Presolved: 502 rows, 502 columns, 2000 nonzeros
Variable types: 2 continuous, 500 integer (500 binary)

Root relaxation: interrupted, 0 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

{'alpha': [0.0,
  0.0,
  0.0,
  2.5,
  2.5,
  2.5,
  0.0,
  2.5,
  0.0,
  0.0,
  2.5,
  2.5,
  0.0,
  2.5,
  0.0,
  0.0,
  2.5,
  2.5,
  0.0,
  2.5,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  2.5,
  0.0,
  2.5,
  2.5,
  2.5,
  2.5,
  0.0,
  0.0,
  2.5,
  0.0,
  0.0,
  2.5,
  2.5,
  2.5,
  2.5,
  2.5,
  0.0,
  2.5,
  0.0,
  0.0,
  2.5,
  0.0,
  0.0,
  0.0,
  2.5,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  2.5,
  2.5,
  0.0,
  2.5,
  2.5,
  0.0,
  0.0,
  2.5,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.

In [126]:
def evaluate(X, y: np.array,beta: float,z: list, radial_kernel:bool=False, w: list=None, X_train=None, y_train=None, alpha: list=None, gamma:float=None):
    N = X.shape[1]
    size = X.shape[0]
    x = np.array([X.iloc[i, :] for i in range(size)])
    y = np.array(y).flatten()
    
    if radial_kernel:
        size_train = X_train.shape[0]
        x_Train = np.array([X_train.iloc[i, :] for i in range(size)])
        global K_
        K = K_(z, gamma)
        y_test = np.array(
            [
                1 if np.sum([alpha[j] * y_train[j] * K(x[i], x_train[j]) for j in range(size_train)]) + beta > 0 else -1
                for i in range(size)
            ]
        )
    else:
        y_test = np.array(
            [
            1 if np.sum([w[j] * z[j] * x[i][j] for j in range(N)]) + beta > 0 else -1
            for i in range(size)
            ]
        )

    pos = y == 1
    neg = y == -1
    tr = y == y_test
    fa = y != y_test
    TP = np.count_nonzero(pos & tr)
    TN = np.count_nonzero(neg & tr)
    FP = np.count_nonzero(pos & fa)
    FN = np.count_nonzero(neg & fa)
    result = {
        "Acc": (TP + TN) / (TP + TN + FP + FN),
        "TPR": TP / (TP + FP),
        "TNR": TN / (TN + FN),
    }
    return result

In [128]:
from itertools import product
from sklearn.model_selection import KFold

def feature_selection(X, y, lambda_:list=[0.5,0.5], radial_kernel:bool=False, folds:int=10, folds2:int=10, C_range:list=[2**i for i in range(-5,6)], gamma_range:list=[2**i for i in range(-5,6)]):
    kf1 = KFold(n_splits=folds, shuffle=True, random_state=42)
    kf2 = KFold(n_splits=folds2, shuffle=True, random_state=42)
    for train_id, test_id in kf1.split(X):
        print(max(train_id), max(test_id), len(X), len(y))
        X_train = X.loc[train_id].reset_index(drop=True)
        y_train = y.loc[train_id].reset_index(drop=True)
        X_test = X.loc[test_id].reset_index(drop=True)
        y_test = y.loc[test_id].reset_index(drop=True)
        
        best_acc = 0
        best_C = None
        best_gamma = None
        if radial_kernel:
            for C, gamma in product(C_range, gamma_range):
                total_acc = 0
                for train_id2, test_id2 in kf2.split(X_train):
                    X_train2 = X_train.loc[train_id2].reset_index(drop=True)
                    y_train2 = y_train.loc[train_id2].reset_index(drop=True)
                    X_test2 = X_train.loc[test_id2].reset_index(drop=True)
                    y_test2 = y_train.loc[test_id2].reset_index(drop=True)
                    result = P1(X=X_train2, y=y_train2, lambda_=lambda_)
                    result2 = P3(X=X_train2, y=y_train2, zk=result["z"], C=C, gamma=gamma, lambda_=lambda_)
                    total_acc += evaluate(X = X_test2, y=y_test2, beta=result2["beta"], radial_kernel=True, X_train=X_train2, y_train=y_train2, alpha=result2["alpha"], z=result["z"], gamma=gamma)["Acc"]
                if total_acc > best_acc:
                    best_acc = total_acc
                    best_C = C
                    best_gamma = gamma
        else:
            # No gamma for linear kernel
            for C in C_range:
                total_acc = 0
                for train_id2, test_id2 in kf2.split(X_train):
                    X_train2 = X_train.loc[train_id2].reset_index(drop=True)
                    y_train2 = y_train.loc[train_id2].reset_index(drop=True)
                    X_test2 = X_train.loc[test_id2].reset_index(drop=True)
                    y_test2 = y_train.loc[test_id2].reset_index(drop=True)
                    result = P1(X=X_train2, y=y_train2, lambda_=lambda_)
                    result2 = P2(X=X_train2, y=y_train2, zk=result["z"], C=C, lambda_=lambda_)
                    total_acc += evaluate(X=X_test2, y= y_test2, beta=result2["beta"], z=result["z"], radial_kernel=False, w=result2["w"])["Acc"]
                if total_acc > best_acc:
                    best_acc = total_acc
                    best_C = C
        result = P1(X=X_train, y=y_train, lambda_=lambda_)
        n_feature = result["z"].count(1)
        if radial_kernel:
            result2 = P3(X=X_train, y=y_train, zk=result["z"], C=best_C, gamma=best_gamma, lambda_=lambda_)
            res = evaluate(X = X_test, y=y_test, beta=result2["beta"], z=result["z"], radial_kernel=True, X_train=X_train, y_train=y_train, alpha=result2["alpha"], gamma=best_gamma)
        else:
            result2 = P2(X=X_train, y=y_train, zk=result["z"], C=best_C, lambda_=lambda_)
            res = evaluate(X=X_test, y= y_test, beta=result2["beta"], z=result["z"], radial_kernel=False, w=result2["w"])
        res["n_feature"] = n_feature
        return res

In [129]:
res = feature_selection(X, y, lambda_=[0.5, 0.5])
res

499 497 500 500
(405, 34)
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2413567 - for non-commercial use only - registered to b0___@ntu.edu.tw
Optimize a model with 475 rows, 474 columns and 4773 nonzeros
Model fingerprint: 0x72ae13b3
Variable types: 35 continuous, 439 integer (439 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 2e+02]
Presolve removed 4 rows and 2 columns
Presolve time: 0.01s
Presolved: 471 rows, 472 columns, 4765 nonzeros
Variable types: 33 continuous, 439 integer (439 binary)

Root relaxation: objective 0.000000e+00, 447 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl | 

{'Acc': 0.74,
 'TPR': 0.6666666666666666,
 'TNR': 0.8076923076923077,
 'n_feature': 1}